In [ ]:
from google.colab import drive
drive.mount('/content/drive')



Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sklearn
import os
import sys
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Conv2D,MaxPool2D,Flatten
from tensorflow.keras.activations import relu,sigmoid
from sklearn.metrics import accuracy_score,confusion_matrix,classification_report
import warnings
warnings.filterwarnings('ignore')
class CUSTOM_MODEL:
  def __init__(self,training_data,testing_data,validation_data):
    try:
      #imageprocessing for all the data
      self.train_formats = ImageDataGenerator(rescale = 1.0/255,
                                   shear_range=0.2,
                                   zoom_range=0.3,
                                   horizontal_flip = True)

      self.labels=os.listdir(training_data)
      self.train_data = self.train_formats.flow_from_directory(training_data,
                                  target_size=(256,256),
                                  classes=self.labels,
                                  class_mode='categorical',
                                  batch_size=10)
      self.valid_data = self.train_formats.flow_from_directory(validation_data,
                                  target_size=(256,256),
                                  classes=self.labels,
                                  class_mode='categorical',
                                  batch_size=10)
      self.test_data = self.train_formats.flow_from_directory(testing_data,
                                  target_size=(256,256),
                                  classes=self.labels,
                                  class_mode='categorical',
                                    batch_size=20)
      self.labels=os.listdir(training_data)
      #print(self.train_data)
    except Exception as e:
      er_type,er_msg,line_no=sys.exc_info()
      print(f'{er_type} <------>{er_msg}<-------->{line_no.tb_lineno}')

  def model_building(self):
    try:
      #model building using custom structure
      self.model = Sequential()

      # first kernel and max poo layers
      self.model.add(Conv2D(128 , kernel_size = (3,3) , input_shape = (int(256),int(256),3) , padding='same',activation='relu'))
      self.model.add(MaxPool2D(pool_size = (2,2)))


      # second kernel and max poo layers
      self.model.add(Conv2D(64 , kernel_size = (3,3), padding='same',activation='relu'))
      self.model.add(MaxPool2D(pool_size = (2,2)))


      # third kernel and max poo layers
      self.model.add(Conv2D(32 , kernel_size = (3,3), padding='same',activation='relu'))
      self.model.add(MaxPool2D(pool_size = (2,2)))


      # forth kernel and max poo layers
      self.model.add(Conv2D(6 , kernel_size = (3,3), padding='same',activation='relu'))
      self.model.add(MaxPool2D(pool_size = (2,2)))


      self.model.add(Flatten()) # one dimensional array


      # above 1d data give to ANN
      self. model.add(Dense(128,activation='relu')) # hiddel layer 1
      self.model.add(Dense(64,activation = 'relu')) # hiddel layer 2


      # output layer
      self.model.add(Dense(34,activation='softmax'))

      print(self.model.summary())


    except Exception as e:
      er_type,er_msg,line_no=sys.exc_info()
      print(f'{er_type} <------>{er_msg}<-------->{line_no.tb_lineno}')

  def training_epoch(self):
    try:
      self.model_building()
      self.model.compile(optimizer='adam',metrics=['accuracy'],loss='categorical_crossentropy')
      self.model.fit(self.train_data,
                     epochs=10,validation_data=self.valid_data)
      self.model.save("/content/drive/MyDrive/TASK_1@VIHARA/FOOD_CLASSIFICATION(200)/custom_model.h5")


    except Exception as e:
      er_type,er_msg,line_no=sys.exc_info()
      print(f'{er_type} <------>{er_msg}<-------->{line_no.tb_lineno}')


  def model_evaluation(self):
    try:
      output_path="/content/drive/MyDrive/TASK_1@VIHARA/FOOD_CLASSIFICATION(200)/ custom_outputs"
      model_path="/content/drive/MyDrive/TASK_1@VIHARA/FOOD_CLASSIFICATION(200)/ custom_outputs/custom_mode.h5"
      self.model_path=model_path
      self.model=load_model(self.model_path)
      self.output_path = os.path.join(output_path, "custom_classification_report.json")
      self.class_names = np.array(list(self.test_data.class_indices.keys()))
      y_true = np.array(self.test_data.classes)
      y_pred_prob = self.model.predict(self.test_data)
      y_pred = np.argmax(y_pred_prob, axis=1)
      conf_matrix = confusion_matrix(y_true, y_pred)

      total_samples = conf_matrix.sum()
      accuracy = (np.sum(np.diag(conf_matrix))) / total_samples

      report = {}

      report["Overall_Accuracy"] = round(accuracy, 4)

      num_classes = len(self.class_names)
      for i in range(num_classes):
          TP = conf_matrix[i, i]
          FP = np.sum(conf_matrix[:, i]) - TP
          FN = np.sum(conf_matrix[i, :]) - TP
          TN = total_samples - (TP + FP + FN + TP)

          precision = TP / (TP + FP) if (TP + FP) > 0 else 0
          recall = TP / (TP + FN) if (TP + FN) > 0 else 0
          f1_score = (2 * precision * recall) / (precision + recall) if (precision + recall) > 0 else 0

          report[self.class_names[i]] = {
              "TP": int(TP),
              "TN": int(TN),
              "FP": int(FP),
              "FN": int(FN),
              "Accuracy": round((TP + TN) / (TP + TN + FP + FN), 4),
              "Precision": round(precision, 4),
              "Recall": round(recall, 4),
              "F1-Score": round(f1_score, 4)
          }

      import json
      with open(self.output_path,'w') as f:
        json.dump(report,f)
    except Exception as e:
      er_type,er_msg,line_no=sys.exc_info()
      print(f'{er_type} <------>{er_msg}<-------->{line_no.tb_lineno}')




if __name__=="__main__":
  training_data="/content/drive/MyDrive/TASK_1@VIHARA/FOOD_CLASSIFICATION(200)/TRAIN_DATASET(150)"
  testing_data="/content/drive/MyDrive/TASK_1@VIHARA/FOOD_CLASSIFICATION(200)/TEST_DATASET(20)"
  validation_data="/content/drive/MyDrive/TASK_1@VIHARA/FOOD_CLASSIFICATION(200)/VALIDATION_DATASET(30)"
  obj=CUSTOM_MODEL(training_data,testing_data,validation_data)
  obj.model_building()


Found 4962 images belonging to 33 classes.
Found 990 images belonging to 33 classes.
Found 660 images belonging to 33 classes.


Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d_12 (Conv2D)                   │ (None, 256, 256, 128)       │           3,584 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_12 (MaxPooling2D)      │ (None, 128, 128, 128)       │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_13 (Conv2D)                   │ (None, 128, 128, 64)        │          73,792 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_13 (MaxPooling2D)      │ (None, 64, 64, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_14 (Conv2D)                   │ (None, 64, 64, 32)          │          18,464 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_14 (MaxPooling2D)      │ (None, 32, 32, 32)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_15 (Conv2D)                   │ (None, 32, 32, 6)           │           1,734 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_15 (MaxPooling2D)      │ (None, 16, 16, 6)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten_4 (Flatten)                  │ (None, 1536)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_10 (Dense)                     │ (None, 128)                 │         196,736 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_11 (Dense)                     │ (None, 64)                  │           8,256 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_12 (Dense)                     │ (None, 34)                  │           2,210 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 304,776 (1.16 MB)

 Trainable params: 304,776 (1.16 MB)

 Non-trainable params: 0 (0.00 B)

None


RESNET





In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sklearn
import os
import sys
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Conv2D,MaxPool2D,Flatten
from tensorflow.keras.activations import relu,sigmoid
from sklearn.metrics import accuracy_score,confusion_matrix,classification_report
import warnings
warnings.filterwarnings('ignore')
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Flatten
from tensorflow.keras.models import Model

class RESNET50_MODEL:
  def __init__(self,training_data,testing_data,validation_data):
    try:

      train_formats = ImageDataGenerator(rescale=1./255,shear_range=0.2,horizontal_flip=True,zoom_range=0.2)
      test_formats= ImageDataGenerator(rescale=1./255)
      self.train_data = train_formats.flow_from_directory(training_data,
                                      target_size=(224,224),batch_size=10,class_mode='categorical')
      self.test_data =test_formats.flow_from_directory(testing_data,
                                      target_size=(224,224),batch_size=10,class_mode='categorical')
      self.valid_data = train_formats.flow_from_directory(validation_data,
                                      target_size=(224,224),batch_size=10,class_mode='categorical')

      self.labels=os.listdir(training_data)
      #print(self.train_data)

    except Exception as e:
        er_type,er_msg,line_no=sys.exc_info()
        print(f'{er_type} <------>{er_msg}<-------->{line_no.tb_lineno}')

  def model_building(self):
   try:
      self.image_size=[224,224]
      self.resnet50 = ResNet50(input_shape=self.image_size + [3], weights='imagenet', include_top=False)
      for layers in self.resnet50.layers:
          layers.trainable = False

      self.x = Flatten()(self.resnet50.output)
      self.predict = Dense(len(self.labels), activation='softmax')(self.x)
      self.model = Model(inputs=self.resnet50.inputs, outputs=self.predict)
      print(self.model.summary())
      self.model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
   except Exception as e:
      er_type, er_msg, line_no = sys.exc_info()
      print(f'Error in line no:{line_no.tb_lineno} and Error Message: {er_msg}')

  def training_epoch(self):
    try:
      self.model.fit(self.train_data,epochs=10,
                     validation_data=self.valid_data)
      self.model.save("/content/drive/MyDrive/TASK_1@VIHARA/FOOD_CLASSIFICATION(200)")
    except Exception as e:
      er_type,er_msg,line_no=sys.exc_info()
      print(f'{er_type} <------>{er_msg}<-------->{line_no.tb_lineno}')

  def model_evaluation(self):

    try:
      output_path="/content/drive/MyDrive/TASK_1@VIHARA/FOOD_CLASSIFICATION(200)/ resnet50_outputs"
      model_path="/content/drive/MyDrive/TASK_1@VIHARA/FOOD_CLASSIFICATION(200)/ resnet50_outputs/resnet_model.h5"
      self.model_path=model_path
      self.model=load_model(self.model_path)
      self.output_path = os.path.join(output_path, "resnet_classification_report.json")
      self.class_names = np.array(list(self.test_data.class_indices.keys()))
      y_true = np.array(self.test_data.classes)
      y_pred_prob = self.model.predict(self.test_data)
      y_pred = np.argmax(y_pred_prob, axis=1)
      conf_matrix = confusion_matrix(y_true, y_pred)

      total_samples = conf_matrix.sum()
      accuracy = (np.sum(np.diag(conf_matrix))) / total_samples

      report = {}

      report["Overall_Accuracy"] = round(accuracy, 4)

      num_classes = len(self.class_names)
      for i in range(num_classes):
          TP = conf_matrix[i, i]
          FP = np.sum(conf_matrix[:, i]) - TP
          FN = np.sum(conf_matrix[i, :]) - TP
          TN = total_samples - (TP + FP + FN + TP)

          precision = TP / (TP + FP) if (TP + FP) > 0 else 0
          recall = TP / (TP + FN) if (TP + FN) > 0 else 0
          f1_score = (2 * precision * recall) / (precision + recall) if (precision + recall) > 0 else 0

          report[self.class_names[i]] = {
              "TP": int(TP),
              "TN": int(TN),
              "FP": int(FP),
              "FN": int(FN),
              "Accuracy": round((TP + TN) / (TP + TN + FP + FN), 4),
              "Precision": round(precision, 4),
              "Recall": round(recall, 4),
              "F1-Score": round(f1_score, 4)
          }

      import json
      with open(self.output_path,'w') as f:
        json.dump(report,f)

    except Exception as e:
      er_type,er_msg,line_no=sys.exc_info()
      print(f'{er_type} <------>{er_msg}<-------->{line_no.tb_lineno}')


if __name__=="__main__":
  try:
    training_data="/content/drive/MyDrive/TASK_1@VIHARA/FOOD_CLASSIFICATION(200)/TRAIN_DATASET(150)"
    testing_data="/content/drive/MyDrive/TASK_1@VIHARA/FOOD_CLASSIFICATION(200)/TEST_DATASET(20)"
    validation_data="/content/drive/MyDrive/TASK_1@VIHARA/FOOD_CLASSIFICATION(200)/VALIDATION_DATASET(30)"

    obj=RESNET50_MODEL(training_data,testing_data,validation_data)
    obj.model_building()

  except Exception as e:
      er_type,er_msg,line_no=sys.exc_info()
      print(f'{er_type} <------>{er_msg}<-------->{line_no.tb_lineno}')




Found 4962 images belonging to 33 classes.
Found 660 images belonging to 33 classes.
Found 990 images belonging to 33 classes.
94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 4s 0us/step


Model: "functional_36"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_3             │ (None, 224, 224, 3)    │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv1_pad (ZeroPadding2D) │ (None, 230, 230, 3)    │              0 │ input_layer_3[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv1_conv (Conv2D)       │ (None, 112, 112, 64)   │          9,472 │ conv1_pad[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv1_bn                  │ (None, 112, 112, 64)   │            256 │ conv1_conv[0][0]       │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv1_relu (Activation)   │ (None, 112, 112, 64)   │              0 │ conv1_bn[0][0]         │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ pool1_pad (ZeroPadding2D) │ (None, 114, 114, 64)   │              0 │ conv1_relu[0][0]       │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ pool1_pool (MaxPooling2D) │ (None, 56, 56, 64)     │              0 │ pool1_pad[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_1_conv       │ (None, 56, 56, 64)     │          4,160 │ pool1_pool[0][0]       │
│ (Conv2D)                  │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_1_bn         │ (None, 56, 56, 64)     │            256 │ conv2_block1_1_conv[0… │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_1_relu       │ (None, 56, 56, 64)     │              0 │ conv2_block1_1_bn[0][… │
│ (Activation)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_2_conv       │ (None, 56, 56, 64)     │         36,928 │ conv2_block1_1_relu[0… │
│ (Conv2D)                  │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_2_bn         │ (None, 56, 56, 64)     │            256 │ conv2_block1_2_conv[0… │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_2_relu       │ (None, 56, 56, 64)     │              0 │ conv2_block1_2_bn[0][… │
│ (Activation)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_0_conv       │ (None, 56, 56, 256)    │         16,640 │ pool1_pool[0][0]       │
│ (Conv2D)                  │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_3_conv       │ (None, 56, 56, 256)    │         16,640 │ conv2_block1_2_relu[0… │
│ (Conv2D)             

 Total params: 26,899,361 (102.61 MB)

 Trainable params: 3,311,649 (12.63 MB)

 Non-trainable params: 23,587,712 (89.98 MB)

None


VGG-16

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sklearn
import os
import sys
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Conv2D,MaxPool2D,Flatten
from tensorflow.keras.activations import relu,sigmoid
from sklearn.metrics import accuracy_score,confusion_matrix,classification_report
import warnings
warnings.filterwarnings('ignore')
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.layers import Flatten
from tensorflow.keras.models import Model

class VGG16_MODEL:
  def __init__(self,training_data,testing_data,validation_data):
    try:

      train_formats = ImageDataGenerator(rescale=1./255,shear_range=0.2,horizontal_flip=True,zoom_range=0.2)
      test_formats= ImageDataGenerator(rescale=1./255)
      self.train_data = train_formats.flow_from_directory(training_data,
                                      target_size=(224,224),batch_size=10,class_mode='categorical')
      self.test_data =test_formats.flow_from_directory(testing_data,
                                      target_size=(224,224),batch_size=20,class_mode='categorical',shuffle=False)
      self.valid_data = train_formats.flow_from_directory(validation_data,
                                      target_size=(224,224),batch_size=10,class_mode='categorical')

      self.labels=os.listdir(training_data)
      #print(self.train_data)

    except Exception as e:
        er_type,er_msg,line_no=sys.exc_info()
        print(f'{er_type} <------>{er_msg}<-------->{line_no.tb_lineno}')

  def model(self):
    try:
      image_size=[224,224]
      vgg16 = VGG16(input_shape=image_size + [3] , weights='imagenet',include_top=False)
      for layers in vgg16.layers:  # VGG16.layers:
        layers.trainable=False
      x =Flatten()(vgg16.output)
      self.predict = Dense(34,activation='softmax')(x)
      self.model = Model(inputs = vgg16.inputs,outputs=self.predict) # VGG16
      print(self.model.summary())
    except Exception as e:
        er_type,er_msg,line_no=sys.exc_info()
        print(f'{er_type} <------>{er_msg}<-------->{line_no.tb_lineno}')


  def training_epoch(self):
    try:
      self.model.compile(loss='categorical_crossentropy',metrics=['accuracy'],optimizer='adam')
      self.model.fit(self.train_data,epochs=10,
                     validation_data=self.valid_data)
      self.model.save("/content/drive/MyDrive/TASK_1@VIHARA/FOOD_CLASSIFICATION(200)/ vgg16_outputs/vgg16_model.h5")
    except Exception as e:
      er_type,er_msg,line_no=sys.exc_info()
      print(f'{er_type} <------>{er_msg}<-------->{line_no.tb_lineno}')

  def model_evaluation(self):
    try:
      output_path="/content/drive/MyDrive/TASK_1@VIHARA/FOOD_CLASSIFICATION(200)/ vgg16_outputs"
      model_path="/content/drive/MyDrive/TASK_1@VIHARA/FOOD_CLASSIFICATION(200)/ vgg16_outputs/vgg16_model.h5"
      self.model_path=model_path
      self.model=load_model(self.model_path)
      self.output_path = os.path.join(output_path, "vgg16_classification_report.json")
      self.class_names = np.array(list(self.test_data.class_indices.keys()))
      y_true = np.array(self.test_data.classes)
      y_pred_prob = self.model.predict(self.test_data)
      y_pred = np.argmax(y_pred_prob, axis=1)
      conf_matrix = confusion_matrix(y_true, y_pred)

      total_samples = conf_matrix.sum()
      accuracy = (np.sum(np.diag(conf_matrix))) / total_samples

      report = {}

      report["Overall_Accuracy"] = round(accuracy, 4)

      num_classes = len(self.class_names)
      for i in range(num_classes):
          TP = conf_matrix[i, i]
          FP = np.sum(conf_matrix[:, i]) - TP
          FN = np.sum(conf_matrix[i, :]) - TP
          TN = total_samples - (TP + FP + FN + TP)

          precision = TP / (TP + FP) if (TP + FP) > 0 else 0
          recall = TP / (TP + FN) if (TP + FN) > 0 else 0
          f1_score = (2 * precision * recall) / (precision + recall) if (precision + recall) > 0 else 0

          report[self.class_names[i]] = {
              "TP": int(TP),
              "TN": int(TN),
              "FP": int(FP),
              "FN": int(FN),
              "Accuracy": round((TP + TN) / (TP + TN + FP + FN), 4),
              "Precision": round(precision, 4),
              "Recall": round(recall, 4),
              "F1-Score": round(f1_score, 4)
          }

      import json
      with open(self.output_path,'w') as f:
        json.dump(report,f)
    except Exception as e:
      er_type,er_msg,line_no = sys.exc_info()
      print(f'Error in line no: {line_no.tb_lineno} and Error Message : {er_msg}')





if __name__=="__main__":
  try:
    training_data="/content/drive/MyDrive/TASK_1@VIHARA/FOOD_CLASSIFICATION(200)/TRAIN_DATASET(150)"
    testing_data="/content/drive/MyDrive/TASK_1@VIHARA/FOOD_CLASSIFICATION(200)/TEST_DATASET(20)"
    validation_data="/content/drive/MyDrive/TASK_1@VIHARA/FOOD_CLASSIFICATION(200)/VALIDATION_DATASET(30)"
    obj=VGG16_MODEL(training_data,testing_data,validation_data)
    obj.model()
  except Exception as e:
      er_type,er_msg,line_no=sys.exc_info()
      print(f'{er_type} <------>{er_msg}<-------->{line_no.tb_lineno}')


Found 4962 images belonging to 33 classes.
Found 660 images belonging to 33 classes.
Found 990 images belonging to 33 classes.
58889256/58889256 ━━━━━━━━━━━━━━━━━━━━ 3s 0us/step


Model: "functional_50"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer_6 (InputLayer)           │ (None, 224, 224, 3)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block1_conv1 (Conv2D)                │ (None, 224, 224, 64)        │           1,792 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block1_conv2 (Conv2D)                │ (None, 224, 224, 64)        │          36,928 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block1_pool (MaxPooling2D)           │ (None, 112, 112, 64)        │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block2_conv1 (Conv2D)                │ (None, 112, 112, 128)       │          73,856 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block2_conv2 (Conv2D)                │ (None, 112, 112, 128)       │         147,584 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block2_pool (MaxPooling2D)           │ (None, 56, 56, 128)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block3_conv1 (Conv2D)                │ (None, 56, 56, 256)         │         295,168 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block3_conv2 (Conv2D)                │ (None, 56, 56, 256)         │         590,080 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block3_conv3 (Conv2D)                │ (None, 56, 56, 256)         │         590,080 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block3_pool (MaxPooling2D)           │ (None, 28, 28, 256)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block4_conv1 (Conv2D)                │ (None, 28, 28, 512)         │       1,180,160 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block4_conv2 (Conv2D)                │ (None, 28, 28, 512)         │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block4_conv3 (Conv2D)                │ (None, 28, 28, 512)         │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block4_pool (MaxPooling2D)           │ (None, 14, 14, 512)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block5_conv1 (Conv2D)                │ (None, 14, 14, 512)         │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block5_conv2 (Conv2D)                │ (None, 14, 14, 512)         │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block5_conv3 (Conv2D)                │ (None, 14, 14, 512)         │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block5_pool (MaxPooling2D)           │ (None, 7, 7, 512)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten_6 (Flatten)                  │ (None, 25088)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_14 (Dense)                     │ (None, 34)                  │         853,026 │
└──────────────────────────────────────┴─────────────────────────────┴──────────────

 Total params: 15,567,714 (59.39 MB)

 Trainable params: 853,026 (3.25 MB)

 Non-trainable params: 14,714,688 (56.13 MB)

None


In [ ]:
from flask import Flask, request, jsonify
from keras.models import load_model
import numpy as np
from tensorflow.keras.preprocessing import image
import json

app = Flask(__name__)

# Load your models (example paths)
vgg16_model = load_model('/content/drive/MyDrive/TASK_1@VIHARA/vgg16_weights_tf_dim_ordering_tf_kernels_notop.h5')
resnet50_model = load_model('/content/drive/MyDrive/TASK_1@VIHARA/resnet50_weights_tf_dim_ordering_tf_kernels_notop.h5')
#custom_model = load_model('path_to_custom_model.h5')

# Load the nutrients and classification report JSON files
with open('nutrients.json', 'r') as f:
    nutrients_data = json.load(f)

# Classification report JSON files
with open('/content/drive/MyDrive/TASK_1@VIHARA/vgg16_weights_tf_dim_ordering_tf_kernels_notop.h5', 'r') as f:
    vgg16_report = json.load(f)

with open('/content/drive/MyDrive/TASK_1@VIHARA/resnet50_weights_tf_dim_ordering_tf_kernels_notop.h5', 'r') as f:
    resnet50_report = json.load(f)

#with open('custom_model_classification_report.json', 'r') as f:
 #   custom_model_report = json.load(f)

# List of food classes (same as in frontend)
food_classes = [
    'Pizza', 'Burger', 'Pasta', 'Ice Cream', 'Cake', 'Sandwich', 'Salad', 'Sushi', 'Soup', 'Steak',
    'Fried Chicken', 'Fish', 'Noodles', 'Donut', 'Hot Dog', 'Wrap', 'Dumplings', 'Tacos', 'Spaghetti', 'Grilled Cheese',
    'Ramen', 'Cupcakes', 'Pie', 'Fries', 'Rice', 'Chicken Wings', 'Pancakes', 'Brownie', 'Muffins', 'Curry', 'Lasagna',
    'Burrito', 'Popcorn', 'Chips', 'Spring Rolls', 'Fajitas'
]

@app.route('/predict', methods=['POST'])
def predict():
    if 'image' not in request.files:
        return jsonify({'error': 'No file part'}), 400

    file = request.files['image']
    if file.filename == '':
        return jsonify({'error': 'No selected file'}), 400

    # Load image and preprocess
    img = image.load_img(file, target_size=(224, 224))  # Assuming the model expects 224x224 input
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension

    model_name = request.args.get('model')

    # Make prediction based on selected model
    if model_name == 'vgg16':
        model = vgg16_model
        classification_report = vgg16_report
    elif model_name == 'resnet50':
        model = resnet50_model
        classification_report = resnet50_report
    elif model_name == 'custom_model':
        model = custom_model
        classification_report = custom_model_report
    else:
        return jsonify({'error': 'Invalid model name'}), 400

    # Predict using the selected model
    pred = np.argmax(model.predict(img_array), axis=-1)[0]

    # Get food name and nutrients based on prediction
    food_name = food_classes[pred]
    nutrients = nutrients_data.get(food_name, [])

    # Return prediction results
    return jsonify({
        'food_name': food_name,
        'vgg16': food_classes[np.argmax(vgg16_model.predict(img_array), axis=-1)[0]],
        'resnet50': food_classes[np.argmax(resnet50_model.predict(img_array), axis=-1)[0]],
#        'custom_model': food_classes[np.argmax(custom_model.predict(img_array), axis=-1)[0]],
        'nutrients': nutrients,
        'classification_report': classification_report
    })

if __name__ == '__main__':
    app.run(debug=True)

ValueError: No model config found in the file at /content/drive/MyDrive/TASK_1@VIHARA/vgg16_weights_tf_dim_ordering_tf_kernels_notop.h5.